# Repeated Holdout Test

**Objectives**
- Load the Model(s)
- Run predictions on the model using the Training/Validation & Holdout/Test* data split using Stratified K-Folds CV (params: k=5 & random_seed=1)
- Record the performance of the models (e.g., accuracy, precision, recall, ROC-AUC, etc.) on every CV instance for use in analysis to determine possible occurence of overfitting.

*The addition of the use of Holdout/Test split here acts as an addition to the original plan of only using Training/Validation data splits.*

**Essentially, this answers the question:** `Is the model overfitted from the training data?`

In [12]:
import pandas as pd

import lightgbm as lgbm
import catboost as catb
 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from joblib import load

import warnings
warnings.filterwarnings("ignore")

In [13]:
DF_LGBM_TB_TRAIN = pd.read_csv('./Dataset/LGBM_TB.csv', low_memory=False) #<== Point these to the appropriate Training+Validation datasets.
DF_LGBM_IB_TRAIN = pd.read_csv('./Dataset/LGBM_IB.csv', low_memory=False)
DF_LGBM_TB_TEST = pd.read_csv('./Dataset/LGBM_TB_Test.csv', low_memory=False)
DF_LGBM_IB_TEST = pd.read_csv('./Dataset/LGBM_IB_Test.csv', low_memory=False)
display(DF_LGBM_TB_TRAIN.head())
display(DF_LGBM_IB_TRAIN.head())
display(DF_LGBM_TB_TEST.head())
display(DF_LGBM_IB_TEST.head())

DF_CATB_TB_TRAIN = pd.read_csv('./Dataset/CATB_TB.csv', low_memory=False)
DF_CATB_IB_TRAIN = pd.read_csv('./Dataset/CATB_IB.csv', low_memory=False)
DF_CATB_TB_TEST = pd.read_csv('./Dataset/CATB_TB_Test.csv', low_memory=False)
DF_CATB_IB_TEST = pd.read_csv('./Dataset/CATB_IB_Test.csv', low_memory=False)

DF_CATB_IB_TRAIN.iloc[:,1:101] = DF_CATB_IB_TRAIN.iloc[:,1:101].astype('str')
DF_CATB_IB_TRAIN.replace("nan", "NaN", inplace=True)
DF_CATB_IB_TEST.iloc[:,1:101] = DF_CATB_IB_TEST.iloc[:,1:101].astype('str')
DF_CATB_IB_TEST.replace("nan", "NaN", inplace=True)

display(DF_CATB_TB_TRAIN.head())
display(DF_CATB_IB_TRAIN.head())
display(DF_CATB_TB_TEST.head())
display(DF_CATB_IB_TEST.head())

,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,112,274,158,215,274,158,215,298,76,...,117,71,297,135,171,215,35,208,56,71
1,1,82,240,117,240,117,240,117,240,117,...,40,209,260,141,260,141,260,141,260,141
2,1,82,86,82,37,70,37,240,117,260,...,198,215,178,215,260,65,260,141,65,20
3,1,82,240,117,240,117,240,117,240,117,...,141,260,141,260,141,260,141,260,141,260
4,1,286,110,172,240,117,240,117,240,117,...,260,141,65,86,99,71,215,240,117,240


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,112,274,158,215,298,76,208,172,117,...,307,307,307,307,307,307,307,307,307,307
1,1,82,240,117,172,16,11,274,158,215,...,307,307,307,307,307,307,307,307,307,307
2,1,82,86,37,70,240,117,260,40,209,...,307,307,307,307,307,307,307,307,307,307
3,1,82,240,117,172,16,11,274,158,215,...,307,307,307,307,307,307,307,307,307,307
4,1,286,110,172,240,117,106,171,260,141,...,307,307,307,307,307,307,307,307,307,307


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,hash,type
0,1,208,172,240,117,240,262,112,123,65,...,274,215,274,215,274,215,274,215,4e270486b92ccff8afa59935ba4f5adc,trojan
1,1,112,274,158,215,274,158,215,298,76,...,297,135,171,215,35,208,56,71,d92d0f24e15384541a0c3c72424fe3a8,trojan
2,1,215,274,158,215,274,158,215,172,117,...,15,240,117,240,117,240,117,172,1dcb9bd8dcdd50f6d07035ea895ecfd1,trojan
3,1,82,240,117,240,117,240,117,240,117,...,208,93,208,16,31,215,108,208,24dd4677c14eb5828bda78749fded6b8,pua
4,1,112,274,158,215,274,158,215,298,76,...,297,135,171,215,35,208,56,71,cc5d38cb80faaf60d8efabecdc04f832,trojan


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,hash,type
0,1,208,172,240,117,262,112,123,65,274,...,307,307,307,307,307,307,307,307,4e270486b92ccff8afa59935ba4f5adc,trojan
1,1,112,274,158,215,298,76,208,172,117,...,307,307,307,307,307,307,307,307,d92d0f24e15384541a0c3c72424fe3a8,trojan
2,1,215,274,158,172,117,198,208,260,257,...,307,307,307,307,307,307,307,307,1dcb9bd8dcdd50f6d07035ea895ecfd1,trojan
3,1,82,240,117,93,16,228,208,198,86,...,307,307,307,307,307,307,307,307,24dd4677c14eb5828bda78749fded6b8,pua
4,1,112,274,158,215,298,76,208,172,117,...,307,307,307,307,307,307,307,307,cc5d38cb80faaf60d8efabecdc04f832,trojan


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,RegOpenKeyExA,NtOpenKey,NtQueryValueKey,NtClose,NtOpenKey,NtQueryValueKey,NtClose,NtQueryAttributesFile,LoadStringA,...,LdrGetProcedureAddress,GetSystemWindowsDirectoryW,NtCreateFile,NtCreateSection,NtMapViewOfSection,NtClose,GetSystemMetrics,NtAllocateVirtualMemory,CreateActCtxW,GetSystemWindowsDirectoryW
1,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegQueryInfoKeyW,RegEnumKeyExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW
2,1,GetSystemTimeAsFileTime,NtCreateMutant,GetSystemTimeAsFileTime,NtOpenKeyEx,NtQueryKey,NtOpenKeyEx,LdrLoadDll,LdrGetProcedureAddress,RegOpenKeyExW,...,GetSystemInfo,NtClose,NtUnmapViewOfSection,NtClose,RegOpenKeyExW,RegCloseKey,RegOpenKeyExW,RegQueryValueExW,RegCloseKey,NtOpenFile
3,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW
4,1,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegOpenKeyExW,RegQueryValueExW,RegCloseKey,NtCreateMutant,GetNativeSystemInfo,GetSystemWindowsDirectoryW,NtClose,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,RegOpenKeyExA,NtOpenKey,NtQueryValueKey,NtClose,NtQueryAttributesFile,LoadStringA,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,GetSystemTimeAsFileTime,NtCreateMutant,NtOpenKeyEx,NtQueryKey,LdrLoadDll,LdrGetProcedureAddress,RegOpenKeyExW,RegQueryInfoKeyW,RegEnumKeyExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,NtOpenSection,NtMapViewOfSection,RegOpenKeyExW,RegQueryValueExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,hash,type
0,1,NtAllocateVirtualMemory,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,NtQuerySystemInformation,RegOpenKeyExA,RegQueryValueExA,RegCloseKey,...,NtOpenKey,NtClose,NtOpenKey,NtClose,NtOpenKey,NtClose,NtOpenKey,NtClose,4e270486b92ccff8afa59935ba4f5adc,trojan
1,1,RegOpenKeyExA,NtOpenKey,NtQueryValueKey,NtClose,NtOpenKey,NtQueryValueKey,NtClose,NtQueryAttributesFile,LoadStringA,...,NtCreateFile,NtCreateSection,NtMapViewOfSection,NtClose,GetSystemMetrics,NtAllocateVirtualMemory,CreateActCtxW,GetSystemWindowsDirectoryW,d92d0f24e15384541a0c3c72424fe3a8,trojan
2,1,NtClose,NtOpenKey,NtQueryValueKey,NtClose,NtOpenKey,NtQueryValueKey,NtClose,LdrGetDllHandle,LdrGetProcedureAddress,...,LookupAccountSidW,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,1dcb9bd8dcdd50f6d07035ea895ecfd1,trojan
3,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,NtAllocateVirtualMemory,GetFileType,NtAllocateVirtualMemory,SetUnhandledExceptionFilter,CoInitializeEx,NtClose,WSAStartup,NtAllocateVirtualMemory,24dd4677c14eb5828bda78749fded6b8,pua
4,1,RegOpenKeyExA,NtOpenKey,NtQueryValueKey,NtClose,NtOpenKey,NtQueryValueKey,NtClose,NtQueryAttributesFile,LoadStringA,...,NtCreateFile,NtCreateSection,NtMapViewOfSection,NtClose,GetSystemMetrics,NtAllocateVirtualMemory,CreateActCtxW,GetSystemWindowsDirectoryW,cc5d38cb80faaf60d8efabecdc04f832,trojan


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,hash,type
0,1,NtAllocateVirtualMemory,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,NtQuerySystemInformation,RegOpenKeyExA,RegQueryValueExA,RegCloseKey,NtOpenKey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4e270486b92ccff8afa59935ba4f5adc,trojan
1,1,RegOpenKeyExA,NtOpenKey,NtQueryValueKey,NtClose,NtQueryAttributesFile,LoadStringA,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d92d0f24e15384541a0c3c72424fe3a8,trojan
2,1,NtClose,NtOpenKey,NtQueryValueKey,LdrGetDllHandle,LdrGetProcedureAddress,GetSystemInfo,NtAllocateVirtualMemory,RegOpenKeyExW,FindFirstFileExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1dcb9bd8dcdd50f6d07035ea895ecfd1,trojan
3,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,GetFileType,SetUnhandledExceptionFilter,NtProtectVirtualMemory,NtAllocateVirtualMemory,GetSystemInfo,NtCreateMutant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24dd4677c14eb5828bda78749fded6b8,pua
4,1,RegOpenKeyExA,NtOpenKey,NtQueryValueKey,NtClose,NtQueryAttributesFile,LoadStringA,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cc5d38cb80faaf60d8efabecdc04f832,trojan


### 0. Defining a helper function for evaluation

In [15]:
import copy
def stratified_kfolds_eval(train_set, test_set, model):
    features = train_set.drop('malware', axis=1)
    labels = train_set['malware']
    kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    print('HOLDOUT FOLD PREDICTIONS')
    print('-------------------------------------------------------')
    ctr = 1
    model_holdout = copy.deepcopy(model)
    for train, holdout in kfolds.split(features, labels):
        X_train = features.iloc[train]
        Y_train = labels.iloc[train]
        model_holdout.fit(X_train, Y_train)
        X_holdout = features.iloc[holdout]
        Y_holdout = labels.iloc[holdout]
        Y_pred = model_holdout.predict(X_holdout)
        print(f"Fold: {ctr}")
        print(classification_report(Y_holdout, Y_pred, digits=4))
        print('-------------------------------------------------------')
        ctr += 1
    
    #comparing each holdout fold result with test split result
    print('\nTEST SPLIT PREDICTIONS')
    print('-------------------------------------------------------')
    Y_pred = model.predict(test_set.iloc[:,1:101])
    print(classification_report(test_set.iloc[:,0], Y_pred, digits=4))
    print('-------------------------------------------------------')

### 1. LightGBM

In [16]:
default_tb = load('Models/LGBM/Default/RYZEN3b_LGBM_TB.model') # <== Point these to the respective .model files
default_ib = load('Models/LGBM/Default/RYZEN3b_LGBM_IB.model')
tuned_tb = load('Models/LGBM/Tuned/TUNED_Manual_RYZEN3b_LGBM_TB.model')
tuned_ib = load('Models/LGBM/Tuned/TUNED_Manual_RYZEN3b_LGBM_IB.model')

print("1. Default LGBM TB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_LGBM_TB_TRAIN, DF_LGBM_TB_TEST, default_tb)

1. Default LGBM TB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9976    0.9918    0.9947      7230
           1     0.9919    0.9976    0.9948      7230

    accuracy                         0.9947     14460
   macro avg     0.9948    0.9947    0.9947     14460
weighted avg     0.9948    0.9947    0.9947     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9981    0.9932    0.9956      7230
           1     0.9933    0.9981    0.9957      7230

    accuracy                         0.9956     14460
   macro avg     0.9957    0.9956    0.9956     14460
weighted avg     0.9957    0.9956    0.9956     14460

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support

 

In [17]:
print("2. Default LGBM IB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_LGBM_IB_TRAIN, DF_LGBM_IB_TEST, default_ib)

2. Default LGBM IB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9982    0.9916    0.9949      7230
           1     0.9916    0.9982    0.9949      7230

    accuracy                         0.9949     14460
   macro avg     0.9949    0.9949    0.9949     14460
weighted avg     0.9949    0.9949    0.9949     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9987    0.9936    0.9962      7230
           1     0.9937    0.9988    0.9962      7230

    accuracy                         0.9962     14460
   macro avg     0.9962    0.9962    0.9962     14460
weighted avg     0.9962    0.9962    0.9962     14460

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support

 

In [18]:
print("3. Tuned LGBM TB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_LGBM_TB_TRAIN, DF_LGBM_TB_TEST, tuned_tb)

3. Tuned LGBM TB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9985    0.9917    0.9951      7230
           1     0.9918    0.9985    0.9951      7230

    accuracy                         0.9951     14460
   macro avg     0.9951    0.9951    0.9951     14460
weighted avg     0.9951    0.9951    0.9951     14460

-------------------------------------------------------


KeyboardInterrupt: 

In [7]:
print("4. Tuned LGBM IB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_LGBM_IB_TRAIN, DF_LGBM_IB_TEST, tuned_ib)

4. Tuned LGBM IB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9996    0.9963    0.9979      7230
           1     0.9963    0.9996    0.9979      7230

    accuracy                         0.9979     14460
   macro avg     0.9979    0.9979    0.9979     14460
weighted avg     0.9979    0.9979    0.9979     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9997    0.9971    0.9984      7230
           1     0.9971    0.9997    0.9984      7230

    accuracy                         0.9984     14460
   macro avg     0.9984    0.9984    0.9984     14460
weighted avg     0.9984    0.9984    0.9984     14460

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support

   

### 2. CatBoost

In [8]:
default_tb = catb.CatBoostClassifier()
default_ib = catb.CatBoostClassifier()
tuned_tb = catb.CatBoostClassifier()
tuned_ib = catb.CatBoostClassifier()
default_tb = default_tb.load_model("./Models/CATB/Default/RYZEN2b_CATB_TB.model", format='json')
default_ib = default_ib.load_model("./Models/CATB/Default/RYZEN2b_CATB_IB.model", format='json')
tuned_tb = tuned_tb.load_model("./Models/CATB/Tuned/TUNED_Manual_RYZEN3b_CATB_TB.model", format='json')
tuned_ib = tuned_ib.load_model("./Models/CATB/Tuned/TUNED_Manual_RYZEN3b_CATB_IB.model", format='json')

print("1. Default CatBoost TB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_CATB_TB_TRAIN, DF_CATB_TB_TEST, default_tb)

1. Default CatBoost TB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9996    0.9945    0.9970      7230
           1     0.9945    0.9996    0.9970      7230

    accuracy                         0.9970     14460
   macro avg     0.9970    0.9970    0.9970     14460
weighted avg     0.9970    0.9970    0.9970     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9999    0.9959    0.9979      7230
           1     0.9959    0.9999    0.9979      7230

    accuracy                         0.9979     14460
   macro avg     0.9979    0.9979    0.9979     14460
weighted avg     0.9979    0.9979    0.9979     14460

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   suppor

In [9]:
print("2. Default CatBoost IB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_CATB_IB_TRAIN, DF_CATB_IB_TEST, default_ib)

2. Default CatBoost IB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9996    0.9961    0.9979      7230
           1     0.9961    0.9996    0.9979      7230

    accuracy                         0.9979     14460
   macro avg     0.9979    0.9979    0.9979     14460
weighted avg     0.9979    0.9979    0.9979     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9996    0.9968    0.9982      7230
           1     0.9968    0.9996    0.9982      7230

    accuracy                         0.9982     14460
   macro avg     0.9982    0.9982    0.9982     14460
weighted avg     0.9982    0.9982    0.9982     14460

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   suppor

In [10]:
print("3. Tuned CatBoost TB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_CATB_TB_TRAIN, DF_CATB_TB_TEST, tuned_tb)

3. Tuned CatBoost TB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9994    0.9947    0.9971      7230
           1     0.9948    0.9994    0.9971      7230

    accuracy                         0.9971     14460
   macro avg     0.9971    0.9971    0.9971     14460
weighted avg     0.9971    0.9971    0.9971     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9997    0.9956    0.9976      7230
           1     0.9956    0.9997    0.9977      7230

    accuracy                         0.9976     14460
   macro avg     0.9977    0.9976    0.9976     14460
weighted avg     0.9977    0.9976    0.9976     14460

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   support


In [11]:
print("4. Default CatBoost IB, 'holdout' from train/validation split vs test split\n")
stratified_kfolds_eval(DF_CATB_IB_TRAIN, DF_CATB_IB_TEST, tuned_ib)

4. Default CatBoost IB, 'holdout' from train/validation split vs test split

HOLDOUT FOLD PREDICTIONS
-------------------------------------------------------
Fold: 1
              precision    recall  f1-score   support

           0     0.9997    0.9938    0.9967      7230
           1     0.9938    0.9997    0.9968      7230

    accuracy                         0.9967     14460
   macro avg     0.9968    0.9967    0.9967     14460
weighted avg     0.9968    0.9967    0.9967     14460

-------------------------------------------------------
Fold: 2
              precision    recall  f1-score   support

           0     0.9996    0.9947    0.9972      7230
           1     0.9948    0.9996    0.9972      7230

    accuracy                         0.9972     14460
   macro avg     0.9972    0.9972    0.9972     14460
weighted avg     0.9972    0.9972    0.9972     14460

-------------------------------------------------------
Fold: 3
              precision    recall  f1-score   suppor